In [0]:
from pyspark.sql.functions import *

In [0]:
%sql
create schema if not exists retail_analytics.silver;

In [0]:
cust_silver_df = spark.read.table("retail_analytics.bronze.customers")

In [0]:
cust_silver_df = (
    cust_silver_df
    .dropDuplicates(["Customer_ID"])
    .withColumn("Email", lower(col("Email")))
    .withColumn("City", trim(col("City")))
    .withColumn("Country", trim(col("Country")))
    .withColumn("Gender", upper(col("Gender")))
    .withColumn("ingestion_date", to_date(col("ingestion_ts")))
)

In [0]:
#Replace null values with "Not available"
cust_silver_df = cust_silver_df.fillna("Not available")

In [0]:
#Gender col cleaning.
cust_silver_df = cust_silver_df.withColumn(
    "Gender",
    when(col("Gender") == "M", "Male")
    .when(col("Gender") == "F", "Female")
    .otherwise(col("Gender"))
)

In [0]:
(cust_silver_df.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable("retail_analytics.silver.customers")
)

In [0]:
display(spark.read.table("retail_analytics.silver.customers").limit(5))